# API(AEMET)

La API de la Agencia Estatal de Meteorología (AEMET) es esencial para enriquecer la tabla que ya tengo tratada del INE debido a su capacidad para proporcionar datos climáticos precisos y actualizados. Estos datos permiten comprender cómo las condiciones climáticas influyen en los patrones de turismo y pernoctaciones, y son fundamentales para la planificación, toma de decisiones y análisis más profundos en el sector turístico y otros campos relacionados.

In [2]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import json
import os
import numpy as np
from pymongo import MongoClient

In [3]:
# utilizo un token de autorización guardado en un archivo para acceder a la API de manera segura y obtener datos climáticos actualizados
aemet = os.getenv("api")

with open('../api.txt', 'r') as file: 
    
    token = file.read()

header = {'Authorization': f"password {token}"}

Voy a extraer los datos mes a mes por diferentes ventajas:
- Granularidad de datos
- Mantenimiento de los datos
- Organización y gestión
- Análisis y comparación más preciso

## Enero 22

In [4]:
#Realizo solicitu GET a la API para obtener los datos diarios de enero 2022 de todas las estaciones.
url = "https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2022-01-01T00%3A00%3A00UTC/fechafin/2022-01-31T00%3A00%3A00UTC/todasestaciones"
querystring = {"api_key": token}

headers = {
    'cache-control': "no-cache"
    }

response = requests.request("GET", url, headers=header, params=querystring)
enero_datos= response.json()
print(response.text)


{
  "descripcion" : "exito",
  "estado" : 200,
  "datos" : "https://opendata.aemet.es/opendata/sh/b338cd98",
  "metadatos" : "https://opendata.aemet.es/opendata/sh/b3aa9d28"
}


In [5]:
#He comprobado que la solicitud ha sido exitosa por lo que hago una segunda solicitud para obtener los datos reales en la key "datos":
url_datos = enero_datos["datos"]
url_datos

'https://opendata.aemet.es/opendata/sh/b338cd98'

In [6]:
response_datos = requests.get(url_datos, headers=header)
response_datos

<Response [200]>

In [7]:
#Convierto la respuesta de la API en formato JSON y accedo a las keys del primer elemento:
datos_enero = response_datos.json()
datos_enero[0].keys()

dict_keys(['fecha', 'indicativo', 'nombre', 'provincia', 'altitud', 'tmed', 'prec', 'tmin', 'horatmin', 'tmax', 'horatmax', 'dir', 'velmedia', 'racha', 'horaracha', 'sol', 'presMax', 'horaPresMax', 'presMin', 'horaPresMin'])

A continuación, tranformo mi lista de dicc en un DataFrame para estructurar la información climática de manera ordenada y eficiente. Esto facilita la manipulación, análisis y visualización de los datos, permitiendo un acceso más rápido y preciso a la información clave, como las temperaturas medias, precipitaciones y fechas.

In [8]:
df_enero = pd.DataFrame(datos_enero)
df_enero.head()


,fecha,indicativo,nombre,provincia,altitud,tmed,prec,tmin,horatmin,tmax,horatmax,dir,velmedia,racha,horaracha,sol,presMax,horaPresMax,presMin,horaPresMin
0,2022-01-01,4358X,DON BENITO,BADAJOZ,273,"13,8","0,0","6,8",02:50,"20,8",15:30,08,"1,4","5,0",09:10,"8,2","996,3",23,"993,8",15
1,2022-01-01,4220X,PUEBLA DE DON RODRIGO,CIUDAD REAL,531,"10,8","0,5","0,8",06:30,"20,8",15:40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-01-01,C447A,TENERIFE NORTE AEROPUERTO,STA. CRUZ DE TENERIFE,632,"19,9","0,0","15,0",08:01,"24,8",11:06,11,"2,8","7,8",14:38,"7,2","954,7",Varias,"950,9",04
3,2022-01-01,6106X,ANTEQUERA,MALAGA,408,"12,6","0,0","5,8",23:50,"19,3",13:00,16,"1,1","9,7",12:10,NaN,"980,1",23,"977,5",Varias
4,2022-01-01,9698U,TALARN,LLEIDA,807,"10,4","0,0","5,0",07:20,"15,7",14:40,17,"1,1","3,3",12:20,NaN,NaN,NaN,NaN,NaN


Una vez en este punto, voy a quedarme con las columnas que necesito y tienen relevancia en mi análisis de turismo. Así simplificaré y optimizaré mi conjunto de datos. Beneficios:

- Reducción de dimensiones
- Mayor claridad
- Eficiencia en el almacenamiento
- Mayor precisión

In [9]:
columnas_interes = ['fecha', 'provincia', 'tmed', 'prec', 'sol', 'tmin', 'tmax']
df_enero = df_enero[columnas_interes]
df_enero.head()

,fecha,provincia,tmed,prec,sol,tmin,tmax
0,2022-01-01,BADAJOZ,"13,8","0,0","8,2","6,8","20,8"
1,2022-01-01,CIUDAD REAL,"10,8","0,5",NaN,"0,8","20,8"
2,2022-01-01,STA. CRUZ DE TENERIFE,"19,9","0,0","7,2","15,0","24,8"
3,2022-01-01,MALAGA,"12,6","0,0",NaN,"5,8","19,3"
4,2022-01-01,LLEIDA,"10,4","0,0",NaN,"5,0","15,7"


In [10]:
df_enero.shape

(7513, 7)

In [11]:
df_enero.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7513 entries, 0 to 7512
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   fecha      7513 non-null   object
 1   provincia  7513 non-null   object
 2   tmed       7301 non-null   object
 3   prec       7300 non-null   object
 4   sol        4303 non-null   object
 5   tmin       7302 non-null   object
 6   tmax       7304 non-null   object
dtypes: object(7)
memory usage: 411.0+ KB


In [12]:
#Realizo una serie de transformaciones en el DF " para reemplazar comas por puntos y convertir los valores en formato de cadena en números de punto flotante (float), además de reemplazar 'Ip' por NaN en la columna 'prec'.

df_enero['tmed'] = df_enero['tmed'].str.replace(',', '.').astype(float)
df_enero['prec'] = df_enero['prec'].replace('Ip', np.nan)
df_enero['prec'] = df_enero['prec'].str.replace(',', '.').astype(float)
df_enero['sol'] = df_enero['sol'].str.replace(',', '.').astype(float)
df_enero['tmin'] = df_enero['tmin'].str.replace(',', '.').astype(float)
df_enero['tmax'] = df_enero['tmax'].str.replace(',', '.').astype(float)

In [13]:
#Convierto la columna "fecha" en datetime:
df_enero['fecha'] = pd.to_datetime(df_enero['fecha'])

In [15]:
df_enero.head()

,fecha,provincia,tmed,prec,sol,tmin,tmax
0,2022-01-01,BADAJOZ,13.8,0.0,8.2,6.8,20.8
1,2022-01-01,CIUDAD REAL,10.8,0.5,NaN,0.8,20.8
2,2022-01-01,STA. CRUZ DE TENERIFE,19.9,0.0,7.2,15.0,24.8
3,2022-01-01,MALAGA,12.6,0.0,NaN,5.8,19.3
4,2022-01-01,LLEIDA,10.4,0.0,NaN,5.0,15.7


In [16]:
# Agrupo por provincia y calculo la media de cada columna:
enero = df_enero.groupby('provincia').agg({
    'tmed': 'mean',
    'prec': 'mean',
    'sol': 'mean',
    'tmin': 'mean',
    'tmax': 'mean'
}).reset_index()

# Renombro las columnas para mayor claridad
enero = enero.rename(columns={
    'tmed': 'media_tmed',
    'prec': 'media_prec',
    'sol': 'media_sol',
    'tmin': 'media_tmin',
    'tmax': 'media_tmax'
})

In [17]:
enero

,provincia,media_tmed,media_prec,media_sol,media_tmin,media_tmax
0,A CORUÑA,10.129289,1.503376,5.338356,6.190377,14.065272
1,ALBACETE,6.384677,0.045968,7.015385,0.026613,12.737097
2,ALICANTE,9.916129,0.433636,6.690323,3.551613,16.283065
3,ALMERIA,11.836585,0.116129,6.777419,7.021138,16.647967
4,ARABA/ALAVA,4.653226,2.538333,5.422581,-1.283871,10.593548
5,ASTURIAS,8.325740,1.619512,4.994170,4.604734,12.047041
6,AVILA,5.700000,0.184000,5.935484,-0.200000,11.592157
7,BADAJOZ,9.021505,0.143011,7.554545,2.715054,15.317742
8,BARCELONA,8.913333,0.241818,6.178378,3.700000,14.126061
9,BIZKAIA,8.689247,3.305691,4.803226,4.669892,12.700000


In [17]:
#eliminar ceuta y melilla
indices_a_eliminar = enero[enero['provincia'].isin(['CEUTA', 'MELILLA'])].index

# Elimino las filas correspondientes a los índices encontrados
enero = enero.drop(indices_a_eliminar)

# Ahora, el DataFrame contendrá las filas sin "CEUTA" y "MELILLA"
enero= enero.reset_index(drop=True)

In [18]:
#Agrego una columna nueva en mi DF enero para exportarlo correctamente en Mongo:
enero['mes'] = 'enero'
enero

,provincia,media_tmed,media_prec,media_sol,media_tmin,media_tmax,mes
0,A CORUÑA,10.129289,1.503376,5.338356,6.190377,14.065272,enero
1,ALBACETE,6.384677,0.045968,7.015385,0.026613,12.737097,enero
2,ALICANTE,9.916129,0.433636,6.690323,3.551613,16.283065,enero
3,ALMERIA,11.836585,0.116129,6.777419,7.021138,16.647967,enero
4,ARABA/ALAVA,4.653226,2.538333,5.422581,-1.283871,10.593548,enero
5,ASTURIAS,8.325740,1.619512,4.994170,4.604734,12.047041,enero
6,AVILA,5.700000,0.184000,5.935484,-0.200000,11.592157,enero
7,BADAJOZ,9.021505,0.143011,7.554545,2.715054,15.317742,enero
8,BARCELONA,8.913333,0.241818,6.178378,3.700000,14.126061,enero
9,BIZKAIA,8.689247,3.305691,4.803226,4.669892,12.700000,enero


Hago una función para aplicar en el resto de meses:

In [19]:
def procesar_mes(df_mes):
    # Reemplazar comas por puntos y convertir las columnas numéricas a tipo float
    df_mes['tmed'] = df_mes['tmed'].str.replace(',', '.').astype(float)
    df_mes['prec'] = df_mes['prec'].replace('Acum', np.nan)
    df_mes['prec'] = df_mes['prec'].str.replace(',', '.').replace('Ip', np.nan).astype(float)
    df_mes['sol'] = df_mes['sol'].str.replace(',', '.').astype(float)
    df_mes['tmin'] = df_mes['tmin'].str.replace(',', '.').astype(float)
    df_mes['tmax'] = df_mes['tmax'].str.replace(',', '.').astype(float)
    
    # Convertir la columna "fecha" en datetime
    df_mes['fecha'] = pd.to_datetime(df_mes['fecha'])
    
    # Agrupar por provincia y calcular la media de cada columna
    df_media_mes = df_mes.groupby('provincia').agg({
        'tmed': 'mean',
        'prec': 'mean',
        'sol': 'mean',
        'tmin': 'mean',
        'tmax': 'mean'
    }).reset_index()
    
    # Renombrar las columnas para mayor claridad
    df_media_mes = df_media_mes.rename(columns={
        'tmed': 'media_tmed',
        'prec': 'media_prec',
        'sol': 'media_sol',
        'tmin': 'media_tmin',
        'tmax': 'media_tmax'
    })
    
    return df_media_mes

In [20]:
#Exporto el DF
enero.to_csv('export/enero22.csv')

In [21]:
# Conecto a la base de datos Mongo
client = MongoClient('localhost', 27017)

In [22]:
enero_data = pd.read_csv('export/enero22.csv')
db = client['turismo']
enero_list = enero_data.to_dict(orient='records')
enero_collection = db['meses']
enero_collection.insert_many(enero_list)

InsertManyResult([ObjectId('6549032f69aed3101fc1e71a'), ObjectId('6549032f69aed3101fc1e71b'), ObjectId('6549032f69aed3101fc1e71c'), ObjectId('6549032f69aed3101fc1e71d'), ObjectId('6549032f69aed3101fc1e71e'), ObjectId('6549032f69aed3101fc1e71f'), ObjectId('6549032f69aed3101fc1e720'), ObjectId('6549032f69aed3101fc1e721'), ObjectId('6549032f69aed3101fc1e722'), ObjectId('6549032f69aed3101fc1e723'), ObjectId('6549032f69aed3101fc1e724'), ObjectId('6549032f69aed3101fc1e725'), ObjectId('6549032f69aed3101fc1e726'), ObjectId('6549032f69aed3101fc1e727'), ObjectId('6549032f69aed3101fc1e728'), ObjectId('6549032f69aed3101fc1e729'), ObjectId('6549032f69aed3101fc1e72a'), ObjectId('6549032f69aed3101fc1e72b'), ObjectId('6549032f69aed3101fc1e72c'), ObjectId('6549032f69aed3101fc1e72d'), ObjectId('6549032f69aed3101fc1e72e'), ObjectId('6549032f69aed3101fc1e72f'), ObjectId('6549032f69aed3101fc1e730'), ObjectId('6549032f69aed3101fc1e731'), ObjectId('6549032f69aed3101fc1e732'), ObjectId('6549032f69aed3101fc1e7

## FEBRERO 22

In [23]:
url2 = "https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2022-02-01T00%3A00%3A00UTC/fechafin/2022-02-28T00%3A00%3A00UTC/todasestaciones"
querystring = {"api_key": token}

headers = {
    'cache-control': "no-cache"
    }

response2 = requests.request("GET", url2, headers=header, params=querystring)
febrero_datos= response2.json()
print(response2.text)

{
  "descripcion" : "exito",
  "estado" : 200,
  "datos" : "https://opendata.aemet.es/opendata/sh/f7f48dd9",
  "metadatos" : "https://opendata.aemet.es/opendata/sh/b3aa9d28"
}


In [24]:
url2_datos= febrero_datos['datos']
url2_datos

'https://opendata.aemet.es/opendata/sh/f7f48dd9'

In [25]:
response2_datos = requests.get(url2_datos, headers=header)
response2_datos

<Response [200]>

In [26]:
datos_febrero= response2_datos.json()
datos_febrero[0].keys()

dict_keys(['fecha', 'indicativo', 'nombre', 'provincia', 'altitud', 'tmed', 'prec', 'tmin', 'horatmin', 'tmax', 'horatmax', 'dir', 'velmedia', 'racha', 'horaracha', 'sol', 'presMax', 'horaPresMax', 'presMin', 'horaPresMin'])

In [27]:
df_febrero= pd.DataFrame(datos_febrero)
df_febrero.head()

,fecha,indicativo,nombre,provincia,altitud,tmed,prec,tmin,horatmin,tmax,horatmax,dir,velmedia,racha,horaracha,sol,presMax,horaPresMax,presMin,horaPresMin
0,2022-02-01,4358X,DON BENITO,BADAJOZ,273,"9,2","0,0","0,8",07:50,"17,7",15:40,09,"0,8","5,0",23:59,"9,5","997,1",09,"994,2",17
1,2022-02-01,4220X,PUEBLA DE DON RODRIGO,CIUDAD REAL,531,"7,4","0,0","-1,7",07:30,"16,5",16:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-02-01,C447A,TENERIFE NORTE AEROPUERTO,STA. CRUZ DE TENERIFE,632,"17,0","2,9","12,6",00:51,"21,3",12:36,NaN,"2,2",NaN,NaN,"0,5","950,2",Varias,"947,7",15
3,2022-02-01,6106X,ANTEQUERA,MALAGA,408,"8,1","0,0","0,4",07:30,"15,8",14:30,31,"1,1","6,4",13:30,NaN,"979,7",24,"976,0",15
4,2022-02-01,9698U,TALARN,LLEIDA,807,"10,2","0,0","2,7",04:20,"17,8",13:50,99,"2,5","8,6",Varias,NaN,NaN,NaN,NaN,NaN


In [28]:
#Aplico la función:

febrero = procesar_mes(df_febrero)
febrero

,provincia,media_tmed,media_prec,media_sol,media_tmin,media_tmax
0,A CORUÑA,10.768304,1.522535,5.409023,6.922321,14.616518
1,ALBACETE,9.515179,0.190909,7.159615,3.148214,15.877679
2,ALICANTE,12.166964,0.072222,7.178571,5.994643,18.336607
3,ALMERIA,13.342727,0.166429,7.192857,8.209091,18.477273
4,ARABA/ALAVA,6.512500,0.366000,5.073214,-0.242857,13.269643
5,ASTURIAS,9.590970,1.451916,5.686567,5.618395,13.566221
6,AVILA,8.616071,0.225000,6.792857,2.158929,15.053571
7,BADAJOZ,11.331548,0.136905,7.321324,4.488095,18.180952
8,BARCELONA,11.382143,0.086429,6.037500,6.654286,16.109286
9,BIZKAIA,10.154737,1.444444,4.192857,5.417895,14.886316


In [29]:
#eliminar ceuta y melilla
indices_a_eliminar = febrero[febrero['provincia'].isin(['CEUTA', 'MELILLA'])].index

# Eliminar las filas correspondientes a los índices encontrados
febrero = febrero.drop(indices_a_eliminar)

# Ahora, el DataFrame contendrá las filas sin "CEUTA" y "MELILLA"
febrero= febrero.reset_index(drop=True)

In [30]:
#Agrego una columna nueva en mi DF enero para exportarlo correctamente en Mongo:
febrero['mes'] = 'febrero'
febrero

,provincia,media_tmed,media_prec,media_sol,media_tmin,media_tmax,mes
0,A CORUÑA,10.768304,1.522535,5.409023,6.922321,14.616518,febrero
1,ALBACETE,9.515179,0.190909,7.159615,3.148214,15.877679,febrero
2,ALICANTE,12.166964,0.072222,7.178571,5.994643,18.336607,febrero
3,ALMERIA,13.342727,0.166429,7.192857,8.209091,18.477273,febrero
4,ARABA/ALAVA,6.512500,0.366000,5.073214,-0.242857,13.269643,febrero
5,ASTURIAS,9.590970,1.451916,5.686567,5.618395,13.566221,febrero
6,AVILA,8.616071,0.225000,6.792857,2.158929,15.053571,febrero
7,BADAJOZ,11.331548,0.136905,7.321324,4.488095,18.180952,febrero
8,BARCELONA,11.382143,0.086429,6.037500,6.654286,16.109286,febrero
9,BIZKAIA,10.154737,1.444444,4.192857,5.417895,14.886316,febrero


In [31]:
febrero.to_csv('export/febrero22.csv')

In [32]:
#Exportación a Mongo:

febrero_data = pd.read_csv('export/febrero22.csv')
db = client['turismo']
febrero_list = febrero_data.to_dict(orient='records')
febrero_collection = db['meses']
febrero_collection.insert_many(febrero_list)

InsertManyResult([ObjectId('6549033f69aed3101fc1e74c'), ObjectId('6549033f69aed3101fc1e74d'), ObjectId('6549033f69aed3101fc1e74e'), ObjectId('6549033f69aed3101fc1e74f'), ObjectId('6549033f69aed3101fc1e750'), ObjectId('6549033f69aed3101fc1e751'), ObjectId('6549033f69aed3101fc1e752'), ObjectId('6549033f69aed3101fc1e753'), ObjectId('6549033f69aed3101fc1e754'), ObjectId('6549033f69aed3101fc1e755'), ObjectId('6549033f69aed3101fc1e756'), ObjectId('6549033f69aed3101fc1e757'), ObjectId('6549033f69aed3101fc1e758'), ObjectId('6549033f69aed3101fc1e759'), ObjectId('6549033f69aed3101fc1e75a'), ObjectId('6549033f69aed3101fc1e75b'), ObjectId('6549033f69aed3101fc1e75c'), ObjectId('6549033f69aed3101fc1e75d'), ObjectId('6549033f69aed3101fc1e75e'), ObjectId('6549033f69aed3101fc1e75f'), ObjectId('6549033f69aed3101fc1e760'), ObjectId('6549033f69aed3101fc1e761'), ObjectId('6549033f69aed3101fc1e762'), ObjectId('6549033f69aed3101fc1e763'), ObjectId('6549033f69aed3101fc1e764'), ObjectId('6549033f69aed3101fc1e7

## MARZO 22

In [33]:
url3 = "https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2022-03-01T00%3A00%3A00UTC/fechafin/2022-03-31T00%3A00%3A00UTC/todasestaciones"
querystring = {"api_key": token}

headers = {
    'cache-control': "no-cache"
    }

response3 = requests.request("GET", url3, headers=header, params=querystring)
marzo_datos= response3.json()
print(response3.text)


{
  "descripcion" : "exito",
  "estado" : 200,
  "datos" : "https://opendata.aemet.es/opendata/sh/bfeac398",
  "metadatos" : "https://opendata.aemet.es/opendata/sh/b3aa9d28"
}


In [34]:
url3_datos = marzo_datos['datos']
url3_datos

'https://opendata.aemet.es/opendata/sh/bfeac398'

In [35]:
response3_datos= requests.get(url3_datos, headers = header)
response3_datos

<Response [200]>

In [36]:
datos_marzo= response3_datos.json()
datos_marzo[0].keys()

dict_keys(['fecha', 'indicativo', 'nombre', 'provincia', 'altitud', 'tmed', 'prec', 'tmin', 'horatmin', 'tmax', 'horatmax', 'dir', 'velmedia', 'racha', 'horaracha', 'sol', 'presMax', 'horaPresMax', 'presMin', 'horaPresMin'])

In [37]:
df_marzo= pd.DataFrame(datos_marzo)
df_marzo.head()

,fecha,indicativo,nombre,provincia,altitud,tmed,prec,tmin,horatmin,tmax,horatmax,dir,velmedia,racha,horaracha,sol,presMax,horaPresMax,presMin,horaPresMin
0,2022-03-01,4358X,DON BENITO,BADAJOZ,273,"13,2","0,0","4,7",07:00,"21,7",13:30,31,"1,1","5,8",16:20,"8,9","995,0",09,"991,1",17
1,2022-03-01,4220X,PUEBLA DE DON RODRIGO,CIUDAD REAL,531,"10,9","0,0","0,7",07:00,"21,1",15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-03-01,C447A,TENERIFE NORTE AEROPUERTO,STA. CRUZ DE TENERIFE,632,"14,1","0,0","11,7",04:26,"16,5",13:33,31,"8,9","20,6",09:26,"5,2","954,7",Varias,"951,6",04
3,2022-03-01,6106X,ANTEQUERA,MALAGA,408,"12,4","0,0","6,7",07:20,"18,1",15:30,15,"1,4","10,8",00:30,NaN,"978,6",10,"974,8",18
4,2022-03-01,9698U,TALARN,LLEIDA,807,"8,8","0,0","1,0",06:30,"16,5",14:40,99,"1,7","6,4",Varias,NaN,NaN,NaN,NaN,NaN


In [38]:
#Aplico la función:
marzo= procesar_mes(df_marzo)

In [39]:
#eliminar ceuta y melilla
indices_a_eliminar = marzo[marzo['provincia'].isin(['CEUTA', 'MELILLA'])].index

# Eliminar las filas correspondientes a los índices encontrados
marzo = marzo.drop(indices_a_eliminar)

# Ahora, el DataFrame contendrá las filas sin "CEUTA" y "MELILLA"
marzo= marzo.reset_index(drop=True)
marzo

,provincia,media_tmed,media_prec,media_sol,media_tmin,media_tmax
0,A CORUÑA,11.490283,3.748899,5.258865,7.617004,15.374597
1,ALBACETE,9.291129,2.700820,2.983929,5.733871,12.842742
2,ALICANTE,12.988710,7.066942,3.702151,9.316935,16.670968
3,ALMERIA,14.205405,2.833077,4.927778,10.381081,18.029054
4,ARABA/ALAVA,8.561290,1.629508,3.272581,3.990323,13.125806
5,ASTURIAS,9.973314,3.297554,4.055696,6.281232,13.670381
6,AVILA,8.612903,4.781356,3.467742,4.943548,12.295161
7,BADAJOZ,12.212366,2.766667,4.734194,7.497849,16.918280
8,BARCELONA,11.885714,2.649664,2.651087,8.777273,14.992857
9,BIZKAIA,12.367742,3.609836,3.096774,8.638710,16.095968


In [40]:
#Agrego una columna nueva en mi DF enero para exportarlo correctamente en Mongo:
marzo['mes'] = 'marzo'
marzo

,provincia,media_tmed,media_prec,media_sol,media_tmin,media_tmax,mes
0,A CORUÑA,11.490283,3.748899,5.258865,7.617004,15.374597,marzo
1,ALBACETE,9.291129,2.700820,2.983929,5.733871,12.842742,marzo
2,ALICANTE,12.988710,7.066942,3.702151,9.316935,16.670968,marzo
3,ALMERIA,14.205405,2.833077,4.927778,10.381081,18.029054,marzo
4,ARABA/ALAVA,8.561290,1.629508,3.272581,3.990323,13.125806,marzo
5,ASTURIAS,9.973314,3.297554,4.055696,6.281232,13.670381,marzo
6,AVILA,8.612903,4.781356,3.467742,4.943548,12.295161,marzo
7,BADAJOZ,12.212366,2.766667,4.734194,7.497849,16.918280,marzo
8,BARCELONA,11.885714,2.649664,2.651087,8.777273,14.992857,marzo
9,BIZKAIA,12.367742,3.609836,3.096774,8.638710,16.095968,marzo


In [ ]:
marzo.to_csv('export/marzo22.csv')

In [43]:
#Exportación a Mongo:

marzo_data = pd.read_csv('export/marzo22.csv')
db = client['turismo']
marzo_list = marzo_data.to_dict(orient='records')
marzo_collection = db['meses']
marzo_collection.insert_many(marzo_list)

InsertManyResult([ObjectId('654903f969aed3101fc1e7b0'), ObjectId('654903f969aed3101fc1e7b1'), ObjectId('654903f969aed3101fc1e7b2'), ObjectId('654903f969aed3101fc1e7b3'), ObjectId('654903f969aed3101fc1e7b4'), ObjectId('654903f969aed3101fc1e7b5'), ObjectId('654903f969aed3101fc1e7b6'), ObjectId('654903f969aed3101fc1e7b7'), ObjectId('654903f969aed3101fc1e7b8'), ObjectId('654903f969aed3101fc1e7b9'), ObjectId('654903f969aed3101fc1e7ba'), ObjectId('654903f969aed3101fc1e7bb'), ObjectId('654903f969aed3101fc1e7bc'), ObjectId('654903f969aed3101fc1e7bd'), ObjectId('654903f969aed3101fc1e7be'), ObjectId('654903f969aed3101fc1e7bf'), ObjectId('654903f969aed3101fc1e7c0'), ObjectId('654903f969aed3101fc1e7c1'), ObjectId('654903f969aed3101fc1e7c2'), ObjectId('654903f969aed3101fc1e7c3'), ObjectId('654903f969aed3101fc1e7c4'), ObjectId('654903f969aed3101fc1e7c5'), ObjectId('654903f969aed3101fc1e7c6'), ObjectId('654903f969aed3101fc1e7c7'), ObjectId('654903f969aed3101fc1e7c8'), ObjectId('654903f969aed3101fc1e7

## ABRIL 22

In [44]:
url4= "https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2022-04-01T00%3A00%3A00UTC/fechafin/2022-04-30T00%3A00%3A00UTC/todasestaciones"
querystring = {"api_key": token}

headers = {
    'cache-control': "no-cache"
    }

response4 = requests.request("GET", url4, headers=header, params=querystring)
abril_datos= response4.json()
print(response4.text)

{
  "descripcion" : "exito",
  "estado" : 200,
  "datos" : "https://opendata.aemet.es/opendata/sh/b4d5ad32",
  "metadatos" : "https://opendata.aemet.es/opendata/sh/b3aa9d28"
}


In [45]:
url4_datos= abril_datos['datos']
url4_datos

'https://opendata.aemet.es/opendata/sh/b4d5ad32'

In [46]:
response4_datos= requests.get(url4_datos, headers=header)
response4_datos

<Response [200]>

In [47]:
datos_abril=response4_datos.json()
datos_abril[0].keys()

dict_keys(['fecha', 'indicativo', 'nombre', 'provincia', 'altitud', 'tmed', 'prec', 'tmin', 'horatmin', 'tmax', 'horatmax', 'dir', 'velmedia', 'racha', 'horaracha', 'sol', 'presMax', 'horaPresMax', 'presMin', 'horaPresMin'])

In [48]:
df_abril= pd.DataFrame(datos_abril)
df_abril.head()

,fecha,indicativo,nombre,provincia,altitud,tmed,prec,tmin,horatmin,tmax,horatmax,dir,velmedia,racha,horaracha,sol,presMax,horaPresMax,presMin,horaPresMin
0,2022-04-01,4358X,DON BENITO,BADAJOZ,273,"12,2","0,0","6,5",06:30,"18,0",15:00,34,"2,2","8,6",16:30,"9,9","986,0",24,"979,7",02
1,2022-04-01,4220X,PUEBLA DE DON RODRIGO,CIUDAD REAL,531,"10,0","0,4","4,0",06:00,"15,9",14:40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-04-01,C447A,TENERIFE NORTE AEROPUERTO,STA. CRUZ DE TENERIFE,632,"14,3","0,0","11,7",Varias,"16,9",12:03,NaN,NaN,NaN,NaN,"5,9",NaN,NaN,NaN,NaN
3,2022-04-01,6106X,ANTEQUERA,MALAGA,408,"11,0","0,0","6,4",06:20,"15,6",16:00,31,"1,7","8,3",13:40,NaN,"967,4",24,"963,8",04
4,2022-04-01,9698U,TALARN,LLEIDA,807,"2,8","0,0","-1,2",23:50,"6,7",14:00,32,"7,2","18,9",08:40,NaN,NaN,NaN,NaN,NaN


In [49]:
#Aplico función:
abril=procesar_mes(df_abril)
abril

,provincia,media_tmed,media_prec,media_sol,media_tmin,media_tmax
0,A CORUÑA,12.187611,2.299561,6.859701,8.303097,16.064159
1,ALBACETE,10.927500,3.810345,6.924074,5.479167,16.370833
2,ALICANTE,14.869167,2.708475,7.886667,9.794167,19.945833
3,ALMERIA,15.366443,1.734014,8.213333,10.883221,19.847651
4,ARABA/ALAVA,9.601667,2.714815,5.845000,3.841667,15.368333
5,ASTURIAS,10.636170,3.420952,6.326244,6.692705,14.572340
6,AVILA,9.119048,2.043902,7.730000,3.645455,14.725000
7,BADAJOZ,13.959551,1.605556,8.265101,7.618539,20.305056
8,BARCELONA,14.297203,1.166216,7.422989,9.644755,18.948252
9,BIZKAIA,12.030088,3.695960,4.696667,8.039823,16.023894


In [51]:
#eliminar ceuta y melilla
indices_a_eliminar = abril[abril['provincia'].isin(['CEUTA', 'MELILLA'])].index

# Eliminar las filas correspondientes a los índices encontrados
abril = abril.drop(indices_a_eliminar)

# Ahora, el DataFrame contendrá las filas sin "CEUTA" y "MELILLA"
abril= abril.reset_index(drop=True)

In [52]:
#Agrego una columna nueva en mi DF abril para exportarlo correctamente en Mongo:
abril['mes'] = 'abril'
abril

,provincia,media_tmed,media_prec,media_sol,media_tmin,media_tmax,mes
0,A CORUÑA,12.187611,2.299561,6.859701,8.303097,16.064159,abril
1,ALBACETE,10.927500,3.810345,6.924074,5.479167,16.370833,abril
2,ALICANTE,14.869167,2.708475,7.886667,9.794167,19.945833,abril
3,ALMERIA,15.366443,1.734014,8.213333,10.883221,19.847651,abril
4,ARABA/ALAVA,9.601667,2.714815,5.845000,3.841667,15.368333,abril
5,ASTURIAS,10.636170,3.420952,6.326244,6.692705,14.572340,abril
6,AVILA,9.119048,2.043902,7.730000,3.645455,14.725000,abril
7,BADAJOZ,13.959551,1.605556,8.265101,7.618539,20.305056,abril
8,BARCELONA,14.297203,1.166216,7.422989,9.644755,18.948252,abril
9,BIZKAIA,12.030088,3.695960,4.696667,8.039823,16.023894,abril


In [53]:
abril.to_csv('export/abril22.csv')

In [54]:
#Exportación a Mongo:

abril_data = pd.read_csv('export/abril22.csv')
db = client['turismo']
abril_list = abril_data.to_dict(orient='records')
abril_collection = db['meses']
abril_collection.insert_many(abril_list)

InsertManyResult([ObjectId('654904b169aed3101fc1e7e2'), ObjectId('654904b169aed3101fc1e7e3'), ObjectId('654904b169aed3101fc1e7e4'), ObjectId('654904b169aed3101fc1e7e5'), ObjectId('654904b169aed3101fc1e7e6'), ObjectId('654904b169aed3101fc1e7e7'), ObjectId('654904b169aed3101fc1e7e8'), ObjectId('654904b169aed3101fc1e7e9'), ObjectId('654904b169aed3101fc1e7ea'), ObjectId('654904b169aed3101fc1e7eb'), ObjectId('654904b169aed3101fc1e7ec'), ObjectId('654904b169aed3101fc1e7ed'), ObjectId('654904b169aed3101fc1e7ee'), ObjectId('654904b169aed3101fc1e7ef'), ObjectId('654904b169aed3101fc1e7f0'), ObjectId('654904b169aed3101fc1e7f1'), ObjectId('654904b169aed3101fc1e7f2'), ObjectId('654904b169aed3101fc1e7f3'), ObjectId('654904b169aed3101fc1e7f4'), ObjectId('654904b169aed3101fc1e7f5'), ObjectId('654904b169aed3101fc1e7f6'), ObjectId('654904b169aed3101fc1e7f7'), ObjectId('654904b169aed3101fc1e7f8'), ObjectId('654904b169aed3101fc1e7f9'), ObjectId('654904b169aed3101fc1e7fa'), ObjectId('654904b169aed3101fc1e7

## MAYO 22

In [55]:
url5= "https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2022-05-01T00%3A00%3A00UTC/fechafin/2022-05-31T00%3A00%3A00UTC/todasestaciones"
querystring = {"api_key": token}

headers = {
    'cache-control': "no-cache"
    }

response5 = requests.request("GET", url5, headers=header, params=querystring)
mayo_datos= response5.json()
print(response5.text)

{
  "descripcion" : "exito",
  "estado" : 200,
  "datos" : "https://opendata.aemet.es/opendata/sh/8611e676",
  "metadatos" : "https://opendata.aemet.es/opendata/sh/b3aa9d28"
}


In [56]:
url5_datos= mayo_datos['datos']
url5_datos

'https://opendata.aemet.es/opendata/sh/8611e676'

In [57]:
response5_datos= requests.get(url5_datos, headers = header)
response5_datos

<Response [200]>

In [58]:
datos_mayo=response5_datos.json()
datos_mayo[0].keys()

dict_keys(['fecha', 'indicativo', 'nombre', 'provincia', 'altitud', 'tmed', 'prec', 'tmin', 'horatmin', 'tmax', 'horatmax', 'dir', 'velmedia', 'racha', 'horaracha', 'sol', 'presMax', 'horaPresMax', 'presMin', 'horaPresMin'])

In [63]:
df_mayo= pd.DataFrame(datos_mayo)
df_mayo.head()

,fecha,indicativo,nombre,provincia,altitud,tmed,prec,tmin,horatmin,tmax,horatmax,dir,velmedia,racha,horaracha,sol,presMax,horaPresMax,presMin,horaPresMin
0,2022-05-01,4358X,DON BENITO,BADAJOZ,273,"20,7","0,0","13,3",23:59,"28,1",Varias,25,"2,8","9,7",13:40,"11,7","988,7",08,"984,2",18
1,2022-05-01,4220X,PUEBLA DE DON RODRIGO,CIUDAD REAL,531,"17,8","0,0","9,2",05:30,"26,3",14:20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-05-01,C447A,TENERIFE NORTE AEROPUERTO,STA. CRUZ DE TENERIFE,632,"18,0","0,0","12,4",23:40,"23,5",13:20,29,"5,3","11,7",21:58,"11,9","952,3",Varias,"950,3",05
3,2022-05-01,6106X,ANTEQUERA,MALAGA,408,"17,6","0,0","9,5",05:00,"25,8",14:30,23,"1,4","6,9",17:10,NaN,"973,6",00,"968,5",19
4,2022-05-01,9698U,TALARN,LLEIDA,807,"14,6","16,8","7,4",04:20,"21,9",12:40,99,"2,8","10,3",Varias,NaN,NaN,NaN,NaN,NaN


In [60]:
#Aplico función:
mayo=procesar_mes(df_mayo)

In [67]:
#eliminar ceuta y melilla
indices_a_eliminar = mayo[mayo['provincia'].isin(['CEUTA', 'MELILLA'])].index

# Eliminar las filas correspondientes a los índices encontrados
mayo = mayo.drop(indices_a_eliminar)

# Ahora, el DataFrame contendrá las filas sin "CEUTA" y "MELILLA"
mayo= mayo.reset_index(drop=True)

In [68]:
#Agrego una columna nueva para especificar el mes y poder filtrar bien en Mongo:
mayo['mes']= 'mayo'
mayo

,provincia,media_tmed,media_prec,media_sol,media_tmin,media_tmax,mes
0,A CORUÑA,16.305809,1.342678,7.578472,12.051867,20.556432,mayo
1,ALBACETE,18.918548,0.447967,11.362500,11.329032,26.507258,mayo
2,ALICANTE,20.262727,0.919008,11.537975,13.944545,26.575455,mayo
3,ALMERIA,20.394483,1.441135,11.187097,15.505517,25.275172,mayo
4,ARABA/ALAVA,15.954839,0.389831,8.048387,8.641935,23.275806,mayo
5,ASTURIAS,15.094395,1.062654,6.226106,11.385841,18.799705,mayo
6,AVILA,17.269767,0.690698,10.432258,10.611111,24.016279,mayo
7,BADAJOZ,21.605914,0.218280,10.985621,13.473118,29.745161,mayo
8,BARCELONA,20.109868,0.815584,9.852174,15.009868,25.203289,mayo
9,BIZKAIA,16.894898,0.865854,6.300000,12.403061,21.397959,mayo


In [69]:
mayo.to_csv('export/mayo22.csv')

In [70]:
#Exportación a Mongo:

mayo_data = pd.read_csv('export/mayo22.csv')
db = client['turismo']
mayo_list = mayo_data.to_dict(orient='records')
mayo_collection = db['meses']
mayo_collection.insert_many(mayo_list)

InsertManyResult([ObjectId('654905e869aed3101fc1e814'), ObjectId('654905e869aed3101fc1e815'), ObjectId('654905e869aed3101fc1e816'), ObjectId('654905e869aed3101fc1e817'), ObjectId('654905e869aed3101fc1e818'), ObjectId('654905e869aed3101fc1e819'), ObjectId('654905e869aed3101fc1e81a'), ObjectId('654905e869aed3101fc1e81b'), ObjectId('654905e869aed3101fc1e81c'), ObjectId('654905e869aed3101fc1e81d'), ObjectId('654905e869aed3101fc1e81e'), ObjectId('654905e869aed3101fc1e81f'), ObjectId('654905e869aed3101fc1e820'), ObjectId('654905e869aed3101fc1e821'), ObjectId('654905e869aed3101fc1e822'), ObjectId('654905e869aed3101fc1e823'), ObjectId('654905e869aed3101fc1e824'), ObjectId('654905e869aed3101fc1e825'), ObjectId('654905e869aed3101fc1e826'), ObjectId('654905e869aed3101fc1e827'), ObjectId('654905e869aed3101fc1e828'), ObjectId('654905e869aed3101fc1e829'), ObjectId('654905e869aed3101fc1e82a'), ObjectId('654905e869aed3101fc1e82b'), ObjectId('654905e869aed3101fc1e82c'), ObjectId('654905e869aed3101fc1e8

## JUNIO 22

In [71]:
url6 = "https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2022-06-01T00%3A00%3A00UTC/fechafin/2022-06-30T00%3A00%3A00UTC/todasestaciones"
querystring = {"api_key": token}

headers = {
    'cache-control': "no-cache"
    }

response6 = requests.request("GET", url6, headers=header, params=querystring)
junio_datos= response6.json()
print(response6.text)

{
  "descripcion" : "exito",
  "estado" : 200,
  "datos" : "https://opendata.aemet.es/opendata/sh/1e04fc76",
  "metadatos" : "https://opendata.aemet.es/opendata/sh/b3aa9d28"
}


In [72]:
url6_datos= junio_datos['datos']
url6_datos

'https://opendata.aemet.es/opendata/sh/1e04fc76'

In [73]:
response6_datos= requests.get(url6_datos, headers=header)
response6_datos

<Response [200]>

In [74]:
datos_junio= response6_datos.json()
datos_junio[0].keys()

dict_keys(['fecha', 'indicativo', 'nombre', 'provincia', 'altitud', 'tmed', 'prec', 'tmin', 'horatmin', 'tmax', 'horatmax', 'dir', 'velmedia', 'racha', 'horaracha', 'sol', 'presMax', 'horaPresMax', 'presMin', 'horaPresMin'])

In [75]:
df_junio= pd.DataFrame(datos_junio)
df_junio.head()

,fecha,indicativo,nombre,provincia,altitud,tmed,prec,tmin,horatmin,tmax,horatmax,dir,velmedia,racha,horaracha,sol,presMax,horaPresMax,presMin,horaPresMin
0,2022-06-01,4358X,DON BENITO,BADAJOZ,273,"23,2","0,0","16,5",05:00,"29,9",17:00,26,"3,1","10,0",15:10,"6,8","984,2",08,"981,1",18
1,2022-06-01,4220X,PUEBLA DE DON RODRIGO,CIUDAD REAL,531,"21,1","0,0","12,2",04:30,"30,0",16:20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-06-01,C447A,TENERIFE NORTE AEROPUERTO,STA. CRUZ DE TENERIFE,632,"16,4","0,0","13,4",22:48,"19,3",15:29,29,"4,2","11,4",08:21,"2,1","948,7",Varias,"944,9",04
3,2022-06-01,6106X,ANTEQUERA,MALAGA,408,"21,3","0,0","13,7",05:00,"28,9",16:00,29,"1,7","7,8",19:00,NaN,"969,3",08,"965,9",Varias
4,2022-06-01,9698U,TALARN,LLEIDA,807,"22,7","0,0","14,6",03:20,"30,8",14:00,24,"2,5","9,2",14:30,NaN,NaN,NaN,NaN,NaN


In [76]:
#Aplico función:
junio= procesar_mes(df_junio)
junio

,provincia,media_tmed,media_prec,media_sol,media_tmin,media_tmax
0,A CORUÑA,17.140529,2.204545,6.298592,13.666520,20.613656
1,ALBACETE,24.450420,0.089655,11.862500,16.366387,32.522689
2,ALICANTE,24.780000,0.255085,12.430000,18.069167,31.484167
3,ALMERIA,23.676154,0.050394,10.983051,18.652308,28.700000
4,ARABA/ALAVA,19.431667,0.510909,7.826667,12.016667,26.850000
5,ASTURIAS,16.899390,2.607166,5.286111,13.256707,20.550000
6,AVILA,19.996667,0.257143,10.603333,13.166667,26.806667
7,BADAJOZ,24.242778,0.043333,11.126846,16.171111,32.310556
8,BARCELONA,24.592617,0.232653,10.644444,19.653691,29.532215
9,BIZKAIA,19.484783,1.889916,5.473333,14.948913,24.016304


In [78]:
#eliminar ceuta y melilla
indices_a_eliminar = junio[junio['provincia'].isin(['CEUTA', 'MELILLA'])].index

# Eliminar las filas correspondientes a los índices encontrados
junio = junio.drop(indices_a_eliminar)

# Ahora, el DataFrame contendrá las filas sin "CEUTA" y "MELILLA"
junio= junio.reset_index(drop=True)

In [80]:
#Añado nueva columna al DF mes de junio:
junio['mes']='junio'

In [82]:
junio.to_csv('export/junio22.csv')

In [84]:
#Exportación a Mongo:

junio_data = pd.read_csv('export/junio22.csv')
db = client['turismo']
junio_list = junio_data.to_dict(orient='records')
junio_collection = db['meses']
junio_collection.insert_many(junio_list)

InsertManyResult([ObjectId('6549085e69aed3101fc1e846'), ObjectId('6549085e69aed3101fc1e847'), ObjectId('6549085e69aed3101fc1e848'), ObjectId('6549085e69aed3101fc1e849'), ObjectId('6549085e69aed3101fc1e84a'), ObjectId('6549085e69aed3101fc1e84b'), ObjectId('6549085e69aed3101fc1e84c'), ObjectId('6549085e69aed3101fc1e84d'), ObjectId('6549085e69aed3101fc1e84e'), ObjectId('6549085e69aed3101fc1e84f'), ObjectId('6549085e69aed3101fc1e850'), ObjectId('6549085e69aed3101fc1e851'), ObjectId('6549085e69aed3101fc1e852'), ObjectId('6549085e69aed3101fc1e853'), ObjectId('6549085e69aed3101fc1e854'), ObjectId('6549085e69aed3101fc1e855'), ObjectId('6549085e69aed3101fc1e856'), ObjectId('6549085e69aed3101fc1e857'), ObjectId('6549085e69aed3101fc1e858'), ObjectId('6549085e69aed3101fc1e859'), ObjectId('6549085e69aed3101fc1e85a'), ObjectId('6549085e69aed3101fc1e85b'), ObjectId('6549085e69aed3101fc1e85c'), ObjectId('6549085e69aed3101fc1e85d'), ObjectId('6549085e69aed3101fc1e85e'), ObjectId('6549085e69aed3101fc1e8

## JULIO 22

In [85]:
url7= "https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2022-07-01T00%3A00%3A00UTC/fechafin/2022-07-31T00%3A00%3A00UTC/todasestaciones"
querystring = {"api_key": token}

headers = {
    'cache-control': "no-cache"
    }

response7 = requests.request("GET", url7, headers=header, params=querystring)
julio_datos= response7.json()
print(response7.text)

{
  "descripcion" : "exito",
  "estado" : 200,
  "datos" : "https://opendata.aemet.es/opendata/sh/5dbc83a2",
  "metadatos" : "https://opendata.aemet.es/opendata/sh/b3aa9d28"
}


In [86]:
url7_datos= julio_datos['datos']
url7_datos

'https://opendata.aemet.es/opendata/sh/5dbc83a2'

In [87]:
response7_datos= requests.get(url7_datos, headers=header)
response7_datos

<Response [200]>

In [88]:
datos_julio=response7_datos.json()
datos_julio[0].keys()

dict_keys(['fecha', 'indicativo', 'nombre', 'provincia', 'altitud', 'tmed', 'prec', 'tmin', 'horatmin', 'tmax', 'horatmax', 'dir', 'velmedia', 'racha', 'horaracha', 'sol', 'presMax', 'horaPresMax', 'presMin', 'horaPresMin'])

In [89]:
df_julio= pd.DataFrame(datos_julio)
df_julio.head()

,fecha,indicativo,nombre,provincia,altitud,tmed,prec,tmin,horatmin,tmax,horatmax,dir,velmedia,racha,horaracha,sol,presMax,horaPresMax,presMin,horaPresMin
0,2022-07-01,4358X,DON BENITO,BADAJOZ,273,"26,4","0,0","16,6",05:40,"36,1",16:30,30,"1,1","5,0",16:50,"11,4","983,4",09,"979,7",19
1,2022-07-01,4220X,PUEBLA DE DON RODRIGO,CIUDAD REAL,531,"24,5","0,0","12,6",05:20,"36,4",16:50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-07-01,C447A,TENERIFE NORTE AEROPUERTO,STA. CRUZ DE TENERIFE,632,"17,2","4,2","14,0",05:21,"20,3",14:45,NaN,NaN,NaN,NaN,"3,2",NaN,NaN,NaN,NaN
3,2022-07-01,6106X,ANTEQUERA,MALAGA,408,"21,4","0,0","14,0",05:40,"28,9",12:10,15,"2,5","11,1",21:10,NaN,"968,5",23,"965,5",04
4,2022-07-01,9698U,TALARN,LLEIDA,807,"20,9","0,0","13,7",04:00,"28,1",16:20,21,"3,1","10,3",Varias,NaN,NaN,NaN,NaN,NaN


In [90]:
#Aplico función:
julio= procesar_mes(df_julio)
julio

,provincia,media_tmed,media_prec,media_sol,media_tmin,media_tmax
0,A CORUÑA,20.661382,0.122785,9.775510,15.865041,25.458943
1,ALBACETE,27.159677,0.350000,12.430645,18.901613,35.417742
2,ALICANTE,27.262903,0.001653,12.211957,20.855645,33.681452
3,ALMERIA,26.990278,0.013571,9.216129,21.835417,32.145833
4,ARABA/ALAVA,20.906452,0.091667,9.348387,12.354839,29.464516
5,ASTURIAS,19.654601,0.745141,8.082178,15.699387,23.615337
6,AVILA,27.105085,0.000000,13.100000,19.005085,35.310000
7,BADAJOZ,29.574194,0.000000,12.850323,20.126344,39.032258
8,BARCELONA,26.762903,0.451240,10.906452,21.455645,32.078226
9,BIZKAIA,21.855645,0.183471,9.216129,16.962903,26.758871


In [92]:
#eliminar ceuta y melilla
indices_a_eliminar = julio[julio['provincia'].isin(['CEUTA', 'MELILLA'])].index

# Eliminar las filas correspondientes a los índices encontrados
julio = julio.drop(indices_a_eliminar)

# Ahora, el DataFrame contendrá las filas sin "CEUTA" y "MELILLA"
julio= julio.reset_index(drop=True)

In [94]:
julio['mes']='julio'

In [97]:
julio.to_csv('export/julio22.csv')

In [98]:
#Exportación a Mongo:

julio_data = pd.read_csv('export/julio22.csv')
db = client['turismo']
julio_list = julio_data.to_dict(orient='records')
julio_collection = db['meses']
julio_collection.insert_many(julio_list)

InsertManyResult([ObjectId('654908e769aed3101fc1e878'), ObjectId('654908e769aed3101fc1e879'), ObjectId('654908e769aed3101fc1e87a'), ObjectId('654908e769aed3101fc1e87b'), ObjectId('654908e769aed3101fc1e87c'), ObjectId('654908e769aed3101fc1e87d'), ObjectId('654908e769aed3101fc1e87e'), ObjectId('654908e769aed3101fc1e87f'), ObjectId('654908e769aed3101fc1e880'), ObjectId('654908e769aed3101fc1e881'), ObjectId('654908e769aed3101fc1e882'), ObjectId('654908e769aed3101fc1e883'), ObjectId('654908e769aed3101fc1e884'), ObjectId('654908e769aed3101fc1e885'), ObjectId('654908e769aed3101fc1e886'), ObjectId('654908e769aed3101fc1e887'), ObjectId('654908e769aed3101fc1e888'), ObjectId('654908e769aed3101fc1e889'), ObjectId('654908e769aed3101fc1e88a'), ObjectId('654908e769aed3101fc1e88b'), ObjectId('654908e769aed3101fc1e88c'), ObjectId('654908e769aed3101fc1e88d'), ObjectId('654908e769aed3101fc1e88e'), ObjectId('654908e769aed3101fc1e88f'), ObjectId('654908e769aed3101fc1e890'), ObjectId('654908e769aed3101fc1e8

## AGOSTO 22

In [99]:
url8="https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2022-08-01T00%3A00%3A00UTC/fechafin/2022-08-31T00%3A00%3A00UTC/todasestaciones"
querystring = {"api_key": token}

headers = {
    'cache-control': "no-cache"
    }

response8 = requests.request("GET", url8, headers=header, params=querystring)
agosto_datos= response8.json()
print(response8.text)

{
  "descripcion" : "exito",
  "estado" : 200,
  "datos" : "https://opendata.aemet.es/opendata/sh/00aee56b",
  "metadatos" : "https://opendata.aemet.es/opendata/sh/b3aa9d28"
}


In [100]:
url8_datos= agosto_datos['datos']
url8_datos

'https://opendata.aemet.es/opendata/sh/00aee56b'

In [101]:
response8_datos= requests.get(url8_datos, headers=header)
response8_datos

<Response [200]>

In [102]:
datos_agosto= response8_datos.json()
datos_agosto[0].keys()

dict_keys(['fecha', 'indicativo', 'nombre', 'provincia', 'altitud', 'tmed', 'prec', 'tmin', 'horatmin', 'tmax', 'horatmax', 'dir', 'velmedia', 'racha', 'horaracha', 'sol', 'presMax', 'horaPresMax', 'presMin', 'horaPresMin'])

In [103]:
df_agosto=pd.DataFrame(datos_agosto)
df_agosto.head()

,fecha,indicativo,nombre,provincia,altitud,tmed,prec,tmin,horatmin,tmax,horatmax,dir,velmedia,racha,horaracha,sol,presMax,horaPresMax,presMin,horaPresMin
0,2022-08-01,4358X,DON BENITO,BADAJOZ,273,"33,8","0,0","25,4",05:50,"42,2",15:10,25,"1,1","7,2",22:30,"11,5","983,5",10,"979,8",18
1,2022-08-01,4220X,PUEBLA DE DON RODRIGO,CIUDAD REAL,531,"31,5","0,0","21,4",04:10,"41,6",15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-08-01,C447A,TENERIFE NORTE AEROPUERTO,STA. CRUZ DE TENERIFE,632,"20,0","0,0","17,0",04:15,"23,0",12:52,99,"8,3","13,3",Varias,"10,5","947,0",Varias,"945,2",19
3,2022-08-01,6106X,ANTEQUERA,MALAGA,408,"27,8","0,0","22,4",05:20,"33,3",15:20,15,"3,3","11,1",13:10,NaN,"969,0",Varias,"966,5",17
4,2022-08-01,9698U,TALARN,LLEIDA,807,"25,2","0,0","17,5",05:00,"33,0",14:50,18,"3,1","10,6",13:50,NaN,NaN,NaN,NaN,NaN


In [104]:
#Aplicar función:
agosto=procesar_mes(df_agosto)

In [105]:
#eliminar ceuta y melilla
indices_a_eliminar = agosto[agosto['provincia'].isin(['CEUTA', 'MELILLA'])].index

# Eliminar las filas correspondientes a los índices encontrados
agosto = agosto.drop(indices_a_eliminar)

# Ahora, el DataFrame contendrá las filas sin "CEUTA" y "MELILLA"
agosto= agosto.reset_index(drop=True)

In [107]:
#Añado columna:
agosto['mes']='agosto'

In [110]:
agosto.to_csv('export/agosto22.csv')

In [111]:
#Exportación a Mongo:

agosto_data = pd.read_csv('export/agosto22.csv')
db = client['turismo']
agosto_list = agosto_data.to_dict(orient='records')
agosto_collection = db['meses']
agosto_collection.insert_many(agosto_list)

InsertManyResult([ObjectId('6549099669aed3101fc1e8aa'), ObjectId('6549099669aed3101fc1e8ab'), ObjectId('6549099669aed3101fc1e8ac'), ObjectId('6549099669aed3101fc1e8ad'), ObjectId('6549099669aed3101fc1e8ae'), ObjectId('6549099669aed3101fc1e8af'), ObjectId('6549099669aed3101fc1e8b0'), ObjectId('6549099669aed3101fc1e8b1'), ObjectId('6549099669aed3101fc1e8b2'), ObjectId('6549099669aed3101fc1e8b3'), ObjectId('6549099669aed3101fc1e8b4'), ObjectId('6549099669aed3101fc1e8b5'), ObjectId('6549099669aed3101fc1e8b6'), ObjectId('6549099669aed3101fc1e8b7'), ObjectId('6549099669aed3101fc1e8b8'), ObjectId('6549099669aed3101fc1e8b9'), ObjectId('6549099669aed3101fc1e8ba'), ObjectId('6549099669aed3101fc1e8bb'), ObjectId('6549099669aed3101fc1e8bc'), ObjectId('6549099669aed3101fc1e8bd'), ObjectId('6549099669aed3101fc1e8be'), ObjectId('6549099669aed3101fc1e8bf'), ObjectId('6549099669aed3101fc1e8c0'), ObjectId('6549099669aed3101fc1e8c1'), ObjectId('6549099669aed3101fc1e8c2'), ObjectId('6549099669aed3101fc1e8

## SEPTIEMBRE 22

In [112]:
url9= "https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2022-09-01T00%3A00%3A00UTC/fechafin/2022-09-30T00%3A00%3A00UTC/todasestaciones"
querystring = {"api_key": token}

headers = {
    'cache-control': "no-cache"
    }

response9 = requests.request("GET", url9, headers=header, params=querystring)
septiembre_datos= response9.json()
print(response9.text)

{
  "descripcion" : "exito",
  "estado" : 200,
  "datos" : "https://opendata.aemet.es/opendata/sh/ce82dd04",
  "metadatos" : "https://opendata.aemet.es/opendata/sh/b3aa9d28"
}


In [113]:
url9_datos= septiembre_datos['datos']
url9_datos

'https://opendata.aemet.es/opendata/sh/ce82dd04'

In [114]:
response9_datos= requests.get(url9_datos, headers=header)
response9_datos

<Response [200]>

In [115]:
datos_septiembre= response9_datos.json()
datos_septiembre[0].keys()

dict_keys(['fecha', 'indicativo', 'nombre', 'provincia', 'altitud', 'tmed', 'prec', 'tmin', 'horatmin', 'tmax', 'horatmax', 'dir', 'velmedia', 'racha', 'horaracha', 'sol', 'presMax', 'horaPresMax', 'presMin', 'horaPresMin'])

In [116]:
df_septiembre= pd.DataFrame(datos_septiembre)
df_septiembre.head()

,fecha,indicativo,nombre,provincia,altitud,tmed,prec,tmin,horatmin,tmax,horatmax,dir,velmedia,racha,horaracha,sol,presMax,horaPresMax,presMin,horaPresMin
0,2022-09-01,4358X,DON BENITO,BADAJOZ,273,"24,1","0,0","16,8",06:00,"31,4",15:00,23,"2,5","8,3",16:00,"8,0","984,9",09,"982,3",Varias
1,2022-09-01,4220X,PUEBLA DE DON RODRIGO,CIUDAD REAL,531,"21,8","0,0","12,2",05:50,"31,3",13:50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-09-01,C447A,TENERIFE NORTE AEROPUERTO,STA. CRUZ DE TENERIFE,632,"22,5","0,1","19,2",05:59,"25,8",11:47,99,"3,3","8,9",Varias,"0,2","949,4",Varias,"946,3",07
3,2022-09-01,6106X,ANTEQUERA,MALAGA,408,"24,5","0,0","16,6",06:50,"32,4",15:30,29,"1,4","8,1",16:10,NaN,"969,0",09,"966,2",15
4,2022-09-01,9698U,TALARN,LLEIDA,807,"20,5","0,0","12,3",04:40,"28,7",15:20,18,"1,7","7,5",14:20,NaN,NaN,NaN,NaN,NaN


In [117]:
#Aplico función:
septiembre= procesar_mes(df_septiembre)
septiembre

,provincia,media_tmed,media_prec,media_sol,media_tmin,media_tmax
0,A CORUÑA,18.676695,3.230568,6.824648,14.812288,22.547458
1,ALBACETE,21.633333,0.981667,8.618182,15.055833,28.206667
2,ALICANTE,24.683333,2.377119,8.538889,18.810833,30.550000
3,ALMERIA,23.904225,0.090345,6.974419,19.039437,28.771127
4,ARABA/ALAVA,17.236667,0.650909,6.813333,9.420000,25.053333
5,ASTURIAS,17.770684,2.340532,6.385024,13.951466,21.591857
6,AVILA,19.152542,1.598246,8.440000,12.530508,25.771186
7,BADAJOZ,22.544068,1.205556,9.116892,15.682486,29.402260
8,BARCELONA,22.934167,0.800847,7.274157,18.038333,27.819167
9,BIZKAIA,19.530000,3.530275,5.516667,14.635556,24.425556


In [118]:
#eliminar ceuta y melilla
indices_a_eliminar = septiembre[septiembre['provincia'].isin(['CEUTA', 'MELILLA'])].index

# Eliminar las filas correspondientes a los índices encontrados
septiembre = septiembre.drop(indices_a_eliminar)

# Ahora, el DataFrame contendrá las filas sin "CEUTA" y "MELILLA"
septiembre= septiembre.reset_index(drop=True)

In [120]:
septiembre['mes']='septiembre'

In [122]:
septiembre.to_csv('export/septiembre22.csv')

In [123]:
#Exportación a Mongo:

septiembre_data = pd.read_csv('export/junio22.csv')
db = client['turismo']
septiembre_list = septiembre_data.to_dict(orient='records')
septiembre_collection = db['meses']
septiembre_collection.insert_many(septiembre_list)

InsertManyResult([ObjectId('65490a2569aed3101fc1e8dc'), ObjectId('65490a2569aed3101fc1e8dd'), ObjectId('65490a2569aed3101fc1e8de'), ObjectId('65490a2569aed3101fc1e8df'), ObjectId('65490a2569aed3101fc1e8e0'), ObjectId('65490a2569aed3101fc1e8e1'), ObjectId('65490a2569aed3101fc1e8e2'), ObjectId('65490a2569aed3101fc1e8e3'), ObjectId('65490a2569aed3101fc1e8e4'), ObjectId('65490a2569aed3101fc1e8e5'), ObjectId('65490a2569aed3101fc1e8e6'), ObjectId('65490a2569aed3101fc1e8e7'), ObjectId('65490a2569aed3101fc1e8e8'), ObjectId('65490a2569aed3101fc1e8e9'), ObjectId('65490a2569aed3101fc1e8ea'), ObjectId('65490a2569aed3101fc1e8eb'), ObjectId('65490a2569aed3101fc1e8ec'), ObjectId('65490a2569aed3101fc1e8ed'), ObjectId('65490a2569aed3101fc1e8ee'), ObjectId('65490a2569aed3101fc1e8ef'), ObjectId('65490a2569aed3101fc1e8f0'), ObjectId('65490a2569aed3101fc1e8f1'), ObjectId('65490a2569aed3101fc1e8f2'), ObjectId('65490a2569aed3101fc1e8f3'), ObjectId('65490a2569aed3101fc1e8f4'), ObjectId('65490a2569aed3101fc1e8

## OCTUBRE 22

In [124]:
url10="https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2022-10-01T00%3A00%3A00UTC/fechafin/2022-10-31T00%3A00%3A00UTC/todasestaciones"
querystring = {"api_key": token}

headers = {
    'cache-control': "no-cache"
    }

response10 = requests.request("GET", url10, headers=header, params=querystring)
octubre_datos= response10.json()
print(response10.text)

{
  "descripcion" : "exito",
  "estado" : 200,
  "datos" : "https://opendata.aemet.es/opendata/sh/10e075b0",
  "metadatos" : "https://opendata.aemet.es/opendata/sh/b3aa9d28"
}


In [125]:
url10_datos=octubre_datos['datos']
url10_datos

'https://opendata.aemet.es/opendata/sh/10e075b0'

In [126]:
response10_datos=requests.get(url10_datos, headers=header)
response10_datos

<Response [200]>

In [127]:
datos_octubre= response10_datos.json()
datos_octubre[0].keys()

dict_keys(['fecha', 'indicativo', 'nombre', 'provincia', 'altitud', 'tmed', 'prec', 'tmin', 'horatmin', 'tmax', 'horatmax', 'dir', 'velmedia', 'racha', 'horaracha', 'sol', 'presMax', 'horaPresMax', 'presMin', 'horaPresMin'])

In [128]:
df_octubre=pd.DataFrame(datos_octubre)
df_octubre.head()

,fecha,indicativo,nombre,provincia,altitud,tmed,prec,tmin,horatmin,tmax,horatmax,dir,velmedia,racha,horaracha,sol,presMax,horaPresMax,presMin,horaPresMin
0,2022-10-01,4358X,DON BENITO,BADAJOZ,273,"19,5","0,0","10,4",05:40,"28,6",15:20,05,"0,8","5,0",14:00,"10,4","993,7",10,"988,7",00
1,2022-10-01,4220X,PUEBLA DE DON RODRIGO,CIUDAD REAL,531,"16,3",NaN,"5,3",06:10,"27,3",14:10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-10-01,C447A,TENERIFE NORTE AEROPUERTO,STA. CRUZ DE TENERIFE,632,"18,4",Ip,"15,6",05:36,"21,1",12:43,32,"3,9","14,4",00:06,"6,9","949,1",Varias,"946,5",17
3,2022-10-01,6106X,ANTEQUERA,MALAGA,408,"15,6","0,0","7,8",06:40,"23,5",13:10,14,"2,8","11,4",18:00,NaN,"977,1",23,"972,1",Varias
4,2022-10-01,9698U,TALARN,LLEIDA,807,"13,8","0,0","5,3",03:30,"22,3",14:30,22,"2,2","6,1",14:30,NaN,NaN,NaN,NaN,NaN


In [129]:
#Aplicar función:
octubre= procesar_mes(df_octubre)

In [130]:
#eliminar ceuta y melilla
indices_a_eliminar = octubre[octubre['provincia'].isin(['CEUTA', 'MELILLA'])].index

# Eliminar las filas correspondientes a los índices encontrados
octubre = octubre.drop(indices_a_eliminar)

# Ahora, el DataFrame contendrá las filas sin "CEUTA" y "MELILLA"
octubre= septiembre.reset_index(drop=True)

In [132]:
#Añado columna 'mes':

octubre['mes']='octubre'

In [133]:
octubre.to_csv('export/octubre22.csv')

In [134]:
#Exportación a Mongo:

octubre_data = pd.read_csv('export/octubre22.csv')
db = client['turismo']
octubre_list = octubre_data.to_dict(orient='records')
octubre_collection = db['meses']
octubre_collection.insert_many(octubre_list)

InsertManyResult([ObjectId('65490b0669aed3101fc1e90e'), ObjectId('65490b0669aed3101fc1e90f'), ObjectId('65490b0669aed3101fc1e910'), ObjectId('65490b0669aed3101fc1e911'), ObjectId('65490b0669aed3101fc1e912'), ObjectId('65490b0669aed3101fc1e913'), ObjectId('65490b0669aed3101fc1e914'), ObjectId('65490b0669aed3101fc1e915'), ObjectId('65490b0669aed3101fc1e916'), ObjectId('65490b0669aed3101fc1e917'), ObjectId('65490b0669aed3101fc1e918'), ObjectId('65490b0669aed3101fc1e919'), ObjectId('65490b0669aed3101fc1e91a'), ObjectId('65490b0669aed3101fc1e91b'), ObjectId('65490b0669aed3101fc1e91c'), ObjectId('65490b0669aed3101fc1e91d'), ObjectId('65490b0669aed3101fc1e91e'), ObjectId('65490b0669aed3101fc1e91f'), ObjectId('65490b0669aed3101fc1e920'), ObjectId('65490b0669aed3101fc1e921'), ObjectId('65490b0669aed3101fc1e922'), ObjectId('65490b0669aed3101fc1e923'), ObjectId('65490b0669aed3101fc1e924'), ObjectId('65490b0669aed3101fc1e925'), ObjectId('65490b0669aed3101fc1e926'), ObjectId('65490b0669aed3101fc1e9

## NOVIEMBRE 22

In [135]:
url11= "https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2022-11-01T00%3A00%3A00UTC/fechafin/2022-11-30T00%3A00%3A00UTC/todasestaciones"
querystring = {"api_key": token}

headers = {
    'cache-control': "no-cache"
    }

response11 = requests.request("GET", url11, headers=header, params=querystring)
noviembre_datos= response11.json()
print(response11.text)

{
  "descripcion" : "exito",
  "estado" : 200,
  "datos" : "https://opendata.aemet.es/opendata/sh/1aaa0a38",
  "metadatos" : "https://opendata.aemet.es/opendata/sh/b3aa9d28"
}


In [136]:
url11_datos= noviembre_datos['datos']
url11_datos

'https://opendata.aemet.es/opendata/sh/1aaa0a38'

In [137]:
response11_datos= requests.get(url11_datos, headers=header)
response11_datos

<Response [200]>

In [138]:
datos_noviembre= response11_datos.json()
datos_noviembre[0].keys()

dict_keys(['fecha', 'indicativo', 'nombre', 'provincia', 'altitud', 'tmed', 'prec', 'tmin', 'horatmin', 'tmax', 'horatmax', 'dir', 'velmedia', 'racha', 'horaracha', 'sol', 'presMax', 'horaPresMax', 'presMin', 'horaPresMin'])

In [139]:
df_noviembre= pd.DataFrame(datos_noviembre)
df_noviembre.head()

,fecha,indicativo,nombre,provincia,altitud,tmed,prec,tmin,horatmin,tmax,horatmax,dir,velmedia,racha,horaracha,sol,presMax,horaPresMax,presMin,horaPresMin
0,2022-11-01,4358X,DON BENITO,BADAJOZ,273,"18,2","0,0","13,8",23:59,"22,5",14:00,34,"0,8","4,4",13:00,"5,9","992,6",Varias,"989,7",00
1,2022-11-01,4220X,PUEBLA DE DON RODRIGO,CIUDAD REAL,531,"17,0","0,0","9,3",07:00,"24,7",14:30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-11-01,C447A,TENERIFE NORTE AEROPUERTO,STA. CRUZ DE TENERIFE,632,"19,6","0,0","14,8",Varias,"24,4",13:57,11,"2,8","6,7",14:17,"9,5","953,3",Varias,"950,6",04
3,2022-11-01,6106X,ANTEQUERA,MALAGA,408,"18,8","0,0","11,3",07:20,"26,3",14:50,31,"1,1","5,6",15:50,NaN,"975,5",09,"972,1",15
4,2022-11-01,9698U,TALARN,LLEIDA,807,"13,0","0,0","8,4",06:00,"17,6",15:40,16,"1,7","4,4",12:40,NaN,NaN,NaN,NaN,NaN


In [140]:
#Aplico la función:
noviembre= procesar_mes(df_noviembre)

In [141]:
#eliminar ceuta y melilla
indices_a_eliminar = noviembre[noviembre['provincia'].isin(['CEUTA', 'MELILLA'])].index

# Eliminar las filas correspondientes a los índices encontrados
noviembre = noviembre.drop(indices_a_eliminar)

# Ahora, el DataFrame contendrá las filas sin "CEUTA" y "MELILLA"
noviembre= noviembre.reset_index(drop=True)

In [143]:
#Añado una columna 'mes':
noviembre['mes']='noviembre'

In [145]:
noviembre.to_csv('export/noviembre22.csv')

In [147]:
#Exportación a Mongo:

noviembre_data = pd.read_csv('export/noviembre22.csv')
db = client['turismo']
noviembre_list = noviembre_data.to_dict(orient='records')
noviembre_collection = db['meses']
noviembre_collection.insert_many(noviembre_list)

InsertManyResult([ObjectId('65490c5c69aed3101fc1e940'), ObjectId('65490c5c69aed3101fc1e941'), ObjectId('65490c5c69aed3101fc1e942'), ObjectId('65490c5c69aed3101fc1e943'), ObjectId('65490c5c69aed3101fc1e944'), ObjectId('65490c5c69aed3101fc1e945'), ObjectId('65490c5c69aed3101fc1e946'), ObjectId('65490c5c69aed3101fc1e947'), ObjectId('65490c5c69aed3101fc1e948'), ObjectId('65490c5c69aed3101fc1e949'), ObjectId('65490c5c69aed3101fc1e94a'), ObjectId('65490c5c69aed3101fc1e94b'), ObjectId('65490c5c69aed3101fc1e94c'), ObjectId('65490c5c69aed3101fc1e94d'), ObjectId('65490c5c69aed3101fc1e94e'), ObjectId('65490c5c69aed3101fc1e94f'), ObjectId('65490c5c69aed3101fc1e950'), ObjectId('65490c5c69aed3101fc1e951'), ObjectId('65490c5c69aed3101fc1e952'), ObjectId('65490c5c69aed3101fc1e953'), ObjectId('65490c5c69aed3101fc1e954'), ObjectId('65490c5c69aed3101fc1e955'), ObjectId('65490c5c69aed3101fc1e956'), ObjectId('65490c5c69aed3101fc1e957'), ObjectId('65490c5c69aed3101fc1e958'), ObjectId('65490c5c69aed3101fc1e9

## DICIEMBRE 22

In [148]:
url12= "https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2022-12-01T00%3A00%3A00UTC/fechafin/2022-12-31T00%3A00%3A00UTC/todasestaciones"
querystring = {"api_key": token}

headers = {
    'cache-control': "no-cache"
    }

response12 = requests.request("GET", url12, headers=header, params=querystring)
diciembre_datos= response12.json()
print(response12.text)

{
  "descripcion" : "exito",
  "estado" : 200,
  "datos" : "https://opendata.aemet.es/opendata/sh/1dbadfee",
  "metadatos" : "https://opendata.aemet.es/opendata/sh/b3aa9d28"
}


In [149]:
url12_datos= diciembre_datos['datos']
url12_datos

'https://opendata.aemet.es/opendata/sh/1dbadfee'

In [150]:
response12_datos= requests.get(url12_datos, headers=header)
response12_datos

<Response [200]>

In [151]:
datos_diciembre= response12_datos.json()
datos_diciembre[0].keys()

dict_keys(['fecha', 'indicativo', 'nombre', 'provincia', 'altitud', 'tmed', 'prec', 'tmin', 'horatmin', 'tmax', 'horatmax', 'dir', 'velmedia', 'racha', 'horaracha', 'sol', 'presMax', 'horaPresMax', 'presMin', 'horaPresMin'])

In [152]:
df_diciembre=pd.DataFrame(datos_diciembre)
df_diciembre.head()

,fecha,indicativo,nombre,provincia,altitud,tmed,prec,tmin,horatmin,tmax,horatmax,dir,velmedia,racha,horaracha,sol,presMax,horaPresMax,presMin,horaPresMin
0,2022-12-01,4358X,DON BENITO,BADAJOZ,273,"12,6","0,0","7,1",23:59,"18,0",14:40,11,"0,8","3,9",09:00,"8,2","982,0",00,"979,8",16
1,2022-12-01,4220X,PUEBLA DE DON RODRIGO,CIUDAD REAL,531,"10,6",NaN,"3,7",23:59,"17,4",13:50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-12-01,C447A,TENERIFE NORTE AEROPUERTO,STA. CRUZ DE TENERIFE,632,"15,3","11,5","11,9",07:46,"18,7",Varias,29,"7,2","13,3",21:45,"7,8","948,1",00,"944,3",15
3,2022-12-01,9698U,TALARN,LLEIDA,807,"2,9","0,0","-1,7",07:20,"7,5",14:50,18,"1,1","3,9",12:30,NaN,NaN,NaN,NaN,NaN
4,2022-12-01,4410X,MÉRIDA,BADAJOZ,228,"10,8","0,0","3,3",23:40,"18,3",14:40,01,"1,1","4,7",11:10,NaN,NaN,NaN,NaN,NaN


In [153]:
#Aplico función:
diciembre=procesar_mes(df_diciembre)

In [154]:
#eliminar ceuta y melilla
indices_a_eliminar = diciembre[diciembre['provincia'].isin(['CEUTA', 'MELILLA'])].index

# Eliminar las filas correspondientes a los índices encontrados
diciembre = diciembre.drop(indices_a_eliminar)

# Ahora, el DataFrame contendrá las filas sin "CEUTA" y "MELILLA"
diciembre= diciembre.reset_index(drop=True)

In [157]:
diciembre['mes']='diciembre'

In [158]:
diciembre.to_csv('export/diciembre22.csv')

In [159]:
#Exportación a Mongo:

diciembre_data = pd.read_csv('export/diciembre22.csv')
db = client['turismo']
diciembre_list = diciembre_data.to_dict(orient='records')
diciembre_collection = db['meses']
diciembre_collection.insert_many(diciembre_list)

InsertManyResult([ObjectId('65490dbb69aed3101fc1e972'), ObjectId('65490dbb69aed3101fc1e973'), ObjectId('65490dbb69aed3101fc1e974'), ObjectId('65490dbb69aed3101fc1e975'), ObjectId('65490dbb69aed3101fc1e976'), ObjectId('65490dbb69aed3101fc1e977'), ObjectId('65490dbb69aed3101fc1e978'), ObjectId('65490dbb69aed3101fc1e979'), ObjectId('65490dbb69aed3101fc1e97a'), ObjectId('65490dbb69aed3101fc1e97b'), ObjectId('65490dbb69aed3101fc1e97c'), ObjectId('65490dbb69aed3101fc1e97d'), ObjectId('65490dbb69aed3101fc1e97e'), ObjectId('65490dbb69aed3101fc1e97f'), ObjectId('65490dbb69aed3101fc1e980'), ObjectId('65490dbb69aed3101fc1e981'), ObjectId('65490dbb69aed3101fc1e982'), ObjectId('65490dbb69aed3101fc1e983'), ObjectId('65490dbb69aed3101fc1e984'), ObjectId('65490dbb69aed3101fc1e985'), ObjectId('65490dbb69aed3101fc1e986'), ObjectId('65490dbb69aed3101fc1e987'), ObjectId('65490dbb69aed3101fc1e988'), ObjectId('65490dbb69aed3101fc1e989'), ObjectId('65490dbb69aed3101fc1e98a'), ObjectId('65490dbb69aed3101fc1e9